In [0]:
%sql
DROP TABLE silver_trip_data

In [0]:
%sql
--first part of silver transformation (CTE inside A CTAS, distance and minutes calculations) --> silver_trip_data
CREATE TABLE silver_trip_data
USING DELTA
AS
WITH T1 AS (
    SELECT
        ride_id,
        rideable_type,
        member_casual,
        start_lat,
        start_lng,
        end_lat,
        end_lng,
        CAST(started_at AS TIMESTAMP) AS trip_start_ts,
        CAST(ended_at AS TIMESTAMP) AS trip_end_ts,
        COALESCE(start_station_name, 'N/A') AS start_station_name,
        COALESCE(start_station_id, 'N/A') AS start_station_id,
        COALESCE(end_station_name, 'N/A') AS end_station_name,
        COALESCE(end_station_id, 'N/A') AS end_station_id
    FROM
        divvy.default.bronze_trip_data
    WHERE 
        _rescued_data IS NULL
        AND started_at IS NOT NULL
        AND ended_at IS NOT NULL
),

Calculated AS (
    SELECT
        *, 
        CASE
            WHEN trip_end_ts > trip_start_ts
            THEN ROUND(TIMESTAMPDIFF(SECOND, trip_start_ts, trip_end_ts) / 60.0, 2)
            ELSE NULL
        END AS Trip_Duration_Min,
        ROUND(
            6371 * 2 * ASIN(
                SQRT(
                    POW(SIN(RADIANS(end_lat - start_lat) / 2), 2) +
                    COS(RADIANS(start_lat)) * COS(RADIANS(end_lat)) *
                    POW(SIN(RADIANS(end_lng - start_lng) / 2), 2)
                )
            ),
            2
        ) AS Trip_Distance_Km,
        CAST(trip_start_ts AS DATE) AS Full_Date
    FROM
        T1
)

SELECT
    T2.trip_start_ts,
    T2.trip_end_ts,
    T2.start_station_name,
    T2.end_station_name,
    T2.ride_id,
    T2.rideable_type,
    T2.member_casual,
    T2.Full_Date,
    T2.start_station_id,
    T2.end_station_id,
    T2.Trip_Duration_Min,
    T2.Trip_Distance_Km,
    T2.start_lat,
    T2.start_lng,
    T2.end_lat,
    T2.end_lng
FROM
    Calculated AS T2
WHERE
    T2.Trip_Distance_Km > 0.05
    AND T2.Trip_Duration_Min IS NOT NULL;

In [0]:
%sql
-- second part of silver transformation --> silver trip data lat and longitude

UPDATE divvy.default.silver_trip_data
SET
    start_lat = ROUND(start_lat, 2), 
    start_lng = ROUND(start_lng, 2), 
    end_lat = ROUND(end_lat, 2), 
    end_lng = ROUND(end_lng, 2)
WHERE
    -- Only update rows that haven't been rounded yet 
    
    CAST(start_lat AS DECIMAL(18, 5)) != ROUND(start_lat, 2)
    OR CAST(start_lng AS DECIMAL(18, 5)) != ROUND(start_lng, 2)
    OR CAST(end_lat AS DECIMAL(18, 5)) != ROUND(end_lat, 2)
    OR CAST(end_lng AS DECIMAL(18, 5)) != ROUND(end_lng, 2);

In [0]:
%sql
-- silver_trip_data_pk
ALTER TABLE silver_trip_data
ALTER COLUMN ride_id SET NOT NULL;

ALTER TABLE silver_trip_data
ADD CONSTRAINT silver_trip_data_pk PRIMARY KEY (ride_id);

In [0]:
%sql
-- Silver Weather Data Transformation

CREATE OR REPLACE TABLE silver_weather_data
USING DELTA
AS
SELECT
    to_timestamp(
        concat(
            CAST(YEAR AS STRING), '-',
            lpad(CAST(MO AS STRING), 2, '0'), '-',
            lpad(CAST(DY AS STRING), 2, '0'), ' ',
            lpad(CAST(HR AS STRING), 2, '0'), ':00:00' 
        ),
        'yyyy-MM-dd HH:mm:ss'
    ) AS weather_ts,
    
    concat(YEAR, lpad(MO, 2, '0'), lpad(DY, 2, '0'), lpad(HR, 2, '0')) AS silver_weather_data_pk,

    CAST(TEMP AS DOUBLE) AS temp_celsius,
    CAST(PRCP AS DOUBLE) AS prcp_mm,
    CAST(HMDT AS DOUBLE) AS hmdt_percent,
    CAST(WND_SPD AS DOUBLE) AS wnd_spd_kph,
    CAST(ATM_PRESS AS DOUBLE) AS atm_press_hpa,
    CAST(
        concat(
            CAST(YEAR AS STRING),
            lpad(CAST(MO AS STRING), 2, '0'),
            lpad(CAST(DY AS STRING), 2, '0')
        ) AS INT
    ) AS Date_Key
    
FROM
    divvy.default.bronze_weather_data
    
WHERE
    _rescued_data IS NULL   
    AND YEAR IS NOT NULL           
    AND TEMP IS NOT NULL

In [0]:
%sql
-- silver_weather_data_pk

ALTER TABLE silver_weather_data
ALTER COLUMN silver_weather_data_pk SET NOT NULL;

ALTER TABLE silver_weather_data
ADD CONSTRAINT silver_weather_data_pk PRIMARY KEY (silver_weather_data_pk)
NOT ENFORCED;